In [15]:
!pip3 install plotly

     |████████████████████████████████| 15.3 MB 11.9 MB/s eta 0:00:01


### Predicting future price for Ethereum

In [101]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [102]:
Ethereum = pd.read_csv('./dataset/coins/coin_Ethereum.csv')
Ethereum.sample(5)

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
194,195,Ethereum,ETH,2016-02-18 23:59:59,4.770570,3.420880,3.774700,4.399130,1.705400e+07,3.393283e+08
2055,2056,Ethereum,ETH,2021-03-24 23:59:59,1740.428200,1570.788001,1678.002539,1593.413457,3.122805e+10,1.835401e+11
913,914,Ethereum,ETH,2018-02-06 23:59:59,796.439026,574.419006,707.737000,793.122009,6.518270e+09,7.729451e+10
184,185,Ethereum,ETH,2016-02-08 23:59:59,3.274280,2.875370,2.973000,3.179790,9.725690e+06,2.444790e+08
1699,1700,Ethereum,ETH,2020-04-02 23:59:59,146.787092,135.732173,135.732173,142.029148,1.532286e+10,1.567396e+10


In [103]:
visualize_Ethereum = cycle(['Open','Close','High','Low','Prediction'])

fig = px.line(Ethereum, x=Ethereum.Date, y=[Ethereum['Open'], Ethereum['Close'], 
                                          Ethereum['High'], Ethereum['Low']],
             labels={'Date': 'Date','value':'Price'})
fig.update_layout(title_text='Ethereum', font_size=15, font_color='black',legend_title_text='Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(visualize_Ethereum)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

In [104]:
Ethereum.shape

(2160, 10)

### Converting datetime to individual columns since LR model could not understand datetime object

In [105]:
import datetime as dt
Ethereum["Date"]=pd.to_datetime(Ethereum["Date"])
Ethereum['Date_year'] = Ethereum["Date"].dt.year
Ethereum['Date_month'] = Ethereum["Date"].dt.month
Ethereum['Date_day'] = Ethereum["Date"].dt.day
Ethereum['Date_hour'] = Ethereum["Date"].dt.hour
Ethereum['Date_minute'] = Ethereum["Date"].dt.minute
Ethereum['Date_seconde'] = Ethereum["Date"].dt.second

In [106]:
Ethereum.drop(["Date"], axis=1, inplace=True)
Ethereum.drop(["Name"],axis=1, inplace=True)
Ethereum.drop(["SNo"],axis=1, inplace=True)
Ethereum.drop(["Symbol"],axis=1, inplace=True)

In [107]:
Ethereum.describe()

,High,Low,Open,Close,Volume,Marketcap,Date_year,Date_month,Date_day,Date_hour,Date_minute,Date_seconde
count,2160.000000,2160.000000,2160.000000,2160.000000,2.160000e+03,2.160000e+03,2160.000000,2160.000000,2160.000000,2160.0,2160.0,2160.0
mean,398.258568,365.592589,382.879899,383.910691,7.057058e+09,4.172084e+10,2018.056944,6.511574,15.732870,23.0,59.0,59.0
std,628.082281,566.611523,599.719862,601.078766,1.064526e+10,6.909184e+10,1.754765,3.473829,8.801313,0.0,0.0,0.0
min,0.482988,0.420897,0.431589,0.434829,1.021280e+05,3.221363e+07,2015.000000,1.000000,1.000000,23.0,59.0,59.0
25%,14.265225,13.190950,13.757600,13.819200,3.825102e+07,1.135576e+09,2017.000000,3.000000,8.000000,23.0,59.0,59.0
50%,205.124631,193.302715,198.425096,198.643691,2.148880e+09,2.070063e+10,2018.000000,6.000000,16.000000,23.0,59.0,59.0
75%,396.494561,375.146804,386.264935,386.435272,9.629136e+09,4.231010e+10,2020.000000,10.000000,23.000000,23.0,59.0,59.0
max,4362.350542,3785.848603,4174.635873,4168.701049,8.448291e+10,4.828819e+11,2021.000000,12.000000,31.000000,23.0,59.0,59.0


In [108]:
X=Ethereum.drop(["Open"], axis=1)
Y=Ethereum["Open"]

In [109]:
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.2)

In [114]:
LR=LinearRegression()

LR.fit(xtrain,ytrain)
ypred=LR.predict(xtest)

In [115]:
ypred

array([-3.33020056e-01,  2.02118399e+02,  1.43095629e+02,  4.57635858e+02,
        1.11287120e+01,  1.21908110e+01,  1.80878444e+02,  2.92404325e+02,
        3.98926037e+02,  2.21259379e+02,  4.52428697e+01,  1.61567161e+02,
        1.36749473e+02,  2.79846075e+02,  3.90534673e+02,  1.12387758e+01,
        9.31522920e+01,  8.32780203e+02, -5.70892264e-01,  3.44864447e+02,
        1.47643375e+02,  1.47323841e+00,  8.51813386e+00,  2.43485228e+02,
        2.98852871e+02,  2.92363591e+02,  2.29982016e+02,  8.52157016e+00,
        8.69618563e+00,  1.49611041e+02,  2.19259800e+03,  1.37850387e+02,
        1.32507492e+02,  3.12580207e+02,  2.87688033e+02,  2.88901010e+02,
        1.76975298e+02,  2.24109473e+03,  2.53088138e+02,  2.17757973e+03,
        1.67207995e+02,  1.60506199e+03, -5.64998668e-01,  1.41388363e+01,
        2.34681703e+02,  1.59885759e+02,  1.62498948e+02,  3.61565191e+02,
        3.55133775e+02,  2.95879232e+02,  3.87768127e+00,  1.01279586e+01,
        1.42325475e+02,  

In [116]:
mean_absolute_error(ytest, ypred)

9.096365411023681

In [117]:
LR.score(xtest,ytest)

0.9985810415902864